In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import math
import datetime
import matplotlib.pyplot as plt
import seaborn as sns

from keras.activations import sigmoid
from keras.models import Model ,load_model
from keras.layers import Input, Dense, ConvLSTM2D, Conv2D, Conv1D, MaxPooling2D, Layer, GlobalAveragePooling2D, Reshape, Flatten, BatchNormalization, Bidirectional
from keras.regularizers import L2
from keras.callbacks import TensorBoard, EarlyStopping, ReduceLROnPlateau
from keras.optimizers.legacy import Adam
from keras.saving import register_keras_serializable
from pandas.tseries.offsets import DateOffset
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import r2_score
from scipy.sparse.linalg import cg


In [ ]:
print(tf.__version__)
print(tf.config.list_physical_devices())
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

In [ ]:
# dirs
DATA_DIR = "./load.csv"

In [ ]:
data = pd.read_csv(DATA_DIR)
data['Timestamp'] = pd.to_datetime(data['Timestamp'], format='%Y/%m/%d %H:%M')
data['Load'] = data['Load'] * 4


In [ ]:
# scaler
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data['Load'].to_numpy().reshape(-1, 1))
data['Load'] = data_scaled

In [ ]:
"""
The dataset columns must be the format below:

  index             Timestamp   Load
      0   20xx-xx-xx xx:xx:xx    xxx
      1                   ...    ...
      2                   ...    ...
                                 ...

"""

In [ ]:
data

In [ ]:
"""
!! parameter settings
n_predict: predict steps
height: final height of the image:
            height * 2 if the n_predict <= width,
            height * 2 + 1 if the n_predict > width
width: width of the image
n_days: use past n days historical time series data as input (number of channel)
n_window_shift: the shift interval of sliding window
"""
n_predict = 96
height = 4
width = 24
n_days_b = 3
n_days_s = 3
n_window_shift = "15min"

In [ ]:
class TimeSeriesImageEncoder():
    def __init__(
            self,
        X: pd.DataFrame,
        n_predict: int,
        height: int,
        width: int,
        n_days_b: int,
        n_days_s: int,
        n_window_shift: str
    ) -> None:
        self.X = X
        self.h = height
        self.m = width
        self.d_b = n_days_b
        self.d_s = n_days_s
        self.shift = n_window_shift
        self.n_predict = n_predict
        self.Lb = self.h * self.m
        self.Ls = math.ceil(self.n_predict / self.m) * self.m
        self.timestamps = self.generate_timestamps()
        print(f"Lb: {self.Lb}")
        print(f"Ls: {self.Ls}")

    def generate_timestamps(self):
        start = self.X['Timestamp'].min() + DateOffset(days=self.d_b)
        end = self.X['Timestamp'].max() - DateOffset(minutes=self.n_predict*15)
        timestamps = pd.date_range(start=start, end=end, freq=self.shift)
        return timestamps
    
    def generate_gaussian_noise(self, length, std_dev=0.15):
        noise = np.random.normal(loc=0.5, scale=std_dev, size=length)
        noise = np.clip(noise, 0, 1)
        # noise = np.zeros(shape=length)
        return pd.DataFrame({"Load": noise})
    
    def make_it_symmetric_3d(self, sets_3d):
        symmetry_training_sets = []
        for slice_2d in np.array(sets_3d):
            reversed_slice_2d = slice_2d[::-1]
            combined_slice_2d = np.concatenate((slice_2d, reversed_slice_2d), axis=0)
            symmetry_training_sets.append(combined_slice_2d)
        return np.array(symmetry_training_sets)
    
    def make_it_symmetric_2d(self, sets_2d):
        reversed_slice_2d = sets_2d[::-1]
        combined_slice_2d = np.concatenate((sets_2d, reversed_slice_2d), axis=0)
        return np.array(combined_slice_2d)
    

    def encode_b(self):
        training_sets = []
        target_sets = []
        self.X_timeseries_flatten = []
        self.X_timestamp = []
        self.y_timestamp = []
        for steps in self.timestamps:
            training_start_b = steps - DateOffset(days=self.d_b-1, hours=23, minutes=45)
            training_end = steps
            target_start = training_end + DateOffset(minutes=15)
            target_end = steps + DateOffset(minutes=(self.n_predict)*15)
            # noise = self.generate_gaussian_noise(length=self.n_predict)
            training_data = self.X[(self.X['Timestamp'] >= training_start_b) & (self.X['Timestamp'] <= training_end)]
            # training_data = pd.concat([training_data, noise], ignore_index=True)
            target_data = self.X[(self.X['Timestamp'] >= target_start) & (self.X['Timestamp'] <= target_end)]
            if not training_data.empty and not target_data.empty:
                self.X_timeseries_flatten.append(training_data['Load'])
                self.X_timestamp.append(training_data['Timestamp'])
                self.y_timestamp.append(target_data['Timestamp'])
                training_reshaped = np.array(training_data['Load']).reshape(self.d_b, self.h, self.m)
                symmetric_3d = self.make_it_symmetric_3d(training_reshaped)
                training_sets.append(symmetric_3d)
                target_reshaped = np.array(target_data['Load']).reshape(math.ceil(self.n_predict/self.m), min(self.n_predict, self.m))
                symmetric_2d = self.make_it_symmetric_2d(target_reshaped)
                target_sets.append(symmetric_2d)
        training_sets = np.array(training_sets)
        target_sets = np.array(target_sets)

        self.X_timeseries_flatten = np.array(self.X_timeseries_flatten)
        self.X_timestamp = np.array(self.X_timestamp)
        self.y_timestamp = np.array(self.y_timestamp)
        return training_sets, target_sets
    
    # def encode_s(self):
    #     training_sets = []
    #     for steps in self.timestamps:
    #         training_subset = []
    #         point = steps - DateOffset(days=self.d_s-1)
    #         training_start = point - DateOffset(minutes=(self.m-1)*15)
    #         # training
    #         for _ in range(self.d_s-1):
    #             training_end = training_start + DateOffset(minutes=(self.m-1)*15)
    #             training_data = self.X[(self.X['Timestamp'] >= training_start) & (self.X['Timestamp'] <= training_end)]
    #             if not training_data.empty:
    #                 symmetric_2d = self.make_it_symmetric_2d(training_data['Load'])
    #                 training_subset.append(symmetric_2d)
    #             training_start = training_start + DateOffset(days=1)
    #         training_end = training_start + DateOffset(minutes=(self.m-self.n_predict-1)*15)
    #         training_data = self.X[(self.X['Timestamp'] >= training_start) & (self.X['Timestamp'] <= training_end)]
    #         noise = self.generate_gaussian_noise(length=self.n_predict)
    #         training_data = pd.concat([training_data, noise], ignore_index=True)
    #         symmetric_2d = self.make_it_symmetric_2d(training_data['Load'])
    #         training_subset.append(symmetric_2d)
    #         training_sets.append(training_subset)
    #     training_sets = np.array(training_sets)
    #     return training_sets
    
    def encode(self):
        training_sets_b, target_sets = self.encode_b()
        # training_sets_s = self.encode_s()
        training_sets_b = np.transpose(training_sets_b, (0, 2, 3, 1))
        # training_sets_s = np.transpose(training_sets_s, (0, 2, 3, 1))
        return training_sets_b, target_sets

In [ ]:
encoder = TimeSeriesImageEncoder(
    X=data,
    n_predict=n_predict,
    height=height,
    width=width,
    n_days_b=n_days_b,
    n_days_s=n_days_s,
    n_window_shift=n_window_shift
)
encoded_Xb, encoded_y = encoder.encode()
X_timeseries = np.copy(encoder.X_timeseries_flatten)
X_timestamp = np.copy(encoder.X_timestamp)
y_timestamp = np.copy(encoder.y_timestamp)

In [ ]:
print(encoded_Xb.shape)
print(encoded_y.shape)

print(X_timeseries.shape)
print(X_timestamp.shape)
print(y_timestamp.shape)

In [ ]:
MONTH_TIME_STEP = math.floor(encoder.timestamps.shape[0] / 24)
X_test_b = []
y_test = []
X_test_b_flatten = []
X_test_b_timestamp = []
y_test_timestamp = []

for i in range(0, 24):
    start = (i+1)*MONTH_TIME_STEP-(192*(i+1))
    end = (i+1)*MONTH_TIME_STEP-(192*i)
    X_test_b.append(encoded_Xb[start:end])
    y_test.append(encoded_y[start:end])
    X_test_b_flatten.append(X_timeseries[start:end])
    X_test_b_timestamp.append(X_timestamp[start:end])
    y_test_timestamp.append(y_timestamp[start:end])


    encoded_Xb = np.concatenate([encoded_Xb[:start], encoded_Xb[end:]])
    encoded_y = np.concatenate([encoded_y[:start], encoded_y[end:]])
    X_timeseries = np.concatenate([X_timeseries[:start], X_timeseries[end:]])
    X_timestamp = np.concatenate([X_timestamp[:start], X_timestamp[end:]])
    y_timestamp = np.concatenate([y_timestamp[:start], y_timestamp[end:]])


In [ ]:
X_test_b = np.concatenate([i for i in X_test_b])
y_test = np.concatenate([i for i in y_test])
X_test_b_flatten = np.concatenate([i for i in X_test_b_flatten])
X_test_b_timestamp = np.concatenate([i for i in X_test_b_timestamp])
y_test_timestamp = np.concatenate([i for i in y_test_timestamp])


In [ ]:
X_train_b = encoded_Xb
y_train = encoded_y

In [ ]:
print(np.array(X_train_b).shape)
print(np.array(X_test_b).shape)
print(np.array(y_train).shape)
print(np.array(y_test).shape)
print(X_test_b_flatten.shape)
print(X_test_b_timestamp.shape)
print(y_test_timestamp.shape)

In [ ]:
@register_keras_serializable('ECALayer')
class ECALayer(Layer):
    def __init__(self, gamma=2, b=1, **kwargs):
        super(ECALayer, self).__init__(**kwargs)
        self.gamma = gamma
        self.b = b

    def build(self, input_shape):
        c = input_shape[-1]
        self.t = max(1, int(abs((tf.math.log(float(c)) / tf.math.log(2.0) + self.b) / self.gamma)))
        self.conv = Conv1D(filters=1, kernel_size=self.t, padding='same', use_bias=False)
        super(ECALayer, self).build(input_shape)

    def call(self, inputs):
        # Global Average Pooling over the spatial dimensions to produce a (batch_size, 1, channels) tensor
        x = GlobalAveragePooling2D()(inputs)
        x = Reshape((1, -1))(x)
        x = self.conv(x)
        x = sigmoid(x)
        x = tf.squeeze(x, axis=1)  # Squeeze to make it (batch_size, channels)
        
        # Multiply weights across channels
        return inputs * x[:, tf.newaxis, tf.newaxis, :]

    def get_config(self):
        config = super(ECALayer, self).get_config()
        config.update({
            'gamma': self.gamma,
            'b': self.b
        })
        return config

In [ ]:
@register_keras_serializable('AverageLayer')
class AverageLayer(Layer):
    def __init__(self, **kwargs):
        super(AverageLayer, self).__init__(**kwargs)

    def call(self, inputs):
        return (inputs[0] + inputs[1]) / 2

In [ ]:
# def create_model(input_shape_b, input_shape_s, num_outputs):
#     inputs_b = Input(shape=input_shape_b)
#     inputs_s = Input(shape=input_shape_s)
#     conv1 = Conv2D(filters=32, kernel_size=8, padding="same", activation="tanh")(inputs_b)
#     conv2 = Conv2D(filters=64, kernel_size=8, padding="same", activation="tanh")(conv1)
#     conv2 = Reshape((1, *conv2.shape[1:]))(conv2)  
#     lstm1 = ConvLSTM2D(filters=96, kernel_size=8, padding="same", activation="tanh", return_sequences=True, dropout=0.3)(conv2)
#     lstm2 = ConvLSTM2D(filters=96, kernel_size=8, padding="same", activation="tanh", return_sequences=False, dropout=0.3)(lstm1)
#     eca1 = ECALayer()(lstm2)
#     conv3 = Conv2D(filters=64, kernel_size=8, padding="same", activation="tanh")(eca1)
#     conv4 = Conv2D(filters=32, kernel_size=8, padding="same", activation="tanh")(conv3)
#     maxpool1 = MaxPooling2D(pool_size=10, padding="same")(conv4)
#     flatten1 = Flatten()(maxpool1)
#     dense1 = Dense(2*num_outputs, activation="linear")(flatten1)
#     outputs1 = Reshape((2, 12))(dense1)

#     conv5 = Conv2D(filters=8, kernel_size=2, padding="same", activation="tanh")(inputs_s)
#     conv6 = Conv2D(filters=16, kernel_size=2, padding="same", activation="tanh")(conv5)
#     conv6 = Reshape((1, *conv6.shape[1:]))(conv6)  
#     lstm3 = ConvLSTM2D(filters=24, kernel_size=2, padding="same", activation="tanh", return_sequences=True, dropout=0.3)(conv6)
#     lstm4 = ConvLSTM2D(filters=24, kernel_size=2, padding="same", activation="tanh", return_sequences=False, dropout=0.3)(lstm3)
#     eca2 = ECALayer()(lstm4)
#     conv7 = Conv2D(filters=16, kernel_size=2, padding="same", activation="tanh")(eca2)
#     conv8 = Conv2D(filters=8, kernel_size=2, padding="same", activation="tanh")(conv7)
#     maxpool2 = MaxPooling2D(pool_size=5, padding="same")(conv8)
#     flatten2 = Flatten()(maxpool2)
#     dense2 = Dense(2*num_outputs, activation="linear")(flatten2)
#     outputs2 = Reshape((2, 12))(dense2)

#     final_output = AverageLayer()([outputs1, outputs2])
#     model = Model(inputs=[inputs_b, inputs_s], outputs=final_output)

#     return model


In [ ]:
def create_model(input_shape_b, encoder):
    height, width = math.ceil(encoder.n_predict / encoder.m) * 2, min(encoder.n_predict, encoder.m)
    inputs_b = Input(shape=input_shape_b)
    conv1 = Conv2D(filters=32, kernel_size=3, padding="same", activation="tanh")(inputs_b)
    conv2 = Conv2D(filters=64, kernel_size=3, padding="same", activation="tanh")(conv1)
    conv2 = Reshape((1, *conv2.shape[1:]))(conv2)
    nor1 = BatchNormalization()(conv2)
    lstm1 = Bidirectional(ConvLSTM2D(filters=96, kernel_size=3, padding="same", activation="tanh", return_sequences=True, dropout=0.0))(nor1)
    nor2 = BatchNormalization()(lstm1)
    lstm2 = Bidirectional(ConvLSTM2D(filters=96, kernel_size=3, padding="same", activation="tanh", return_sequences=False, dropout=0.0))(nor2)
    nor3 = BatchNormalization()(lstm2)
    eca1 = ECALayer()(nor3)
    nor4 = BatchNormalization()(eca1)
    conv3 = Conv2D(filters=64, kernel_size=3, padding="same", activation="tanh")(nor4)
    conv4 = Conv2D(filters=32, kernel_size=3, padding="same", activation="tanh")(conv3)
    nor5 = BatchNormalization()(conv4)
    maxpool1 = MaxPooling2D(pool_size=10, padding="same")(nor5)
    flatten1 = Flatten()(maxpool1)
    dense1 = Dense(height*width, activation="linear")(flatten1)
    outputs = Reshape((height, width))(dense1)
    model = Model(inputs=inputs_b, outputs=outputs)

    return model

In [ ]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

%load_ext tensorboard

tensorboard_callback = TensorBoard(logdir, histogram_freq=1)
early_stopping_callback = EarlyStopping(monitor="loss", patience=10, min_delta=5e-5)
reduce_lr_callback = ReduceLROnPlateau(monitor="loss", factor=0.3, patience=5, verbose=1, min_lr=1e-7)
callbacks=[tensorboard_callback, early_stopping_callback, reduce_lr_callback]
model = create_model(input_shape_b=X_train_b.shape[1:], encoder=encoder)
model.compile(optimizer=Adam(learning_rate=5e-5), loss="mse")
model.summary()

In [ ]:
history = model.fit(
    X_train_b,
    y_train,
    verbose=1,
    epochs=120,
    batch_size=96,
    callbacks=[tensorboard_callback, early_stopping_callback, reduce_lr_callback]
)
 


In [ ]:
plt.plot(history.history['loss'], label='Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
y_pred = model.predict([X_test_b])

In [ ]:
def sum_np(matrix):
    # Calculate the number of pairs
    n_pairs = len(matrix) // 2
    
    # Initialize a list to hold the sums
    sums = []
    
    # Iterate through pairs of rows: first-last, second-second last, etc.
    for i in range(n_pairs):
        sums.append(list(map(sum, zip(matrix[i], matrix[-(i + 1)]))))
        
    # If there's an odd number of rows, include the middle row
    if len(matrix) % 2 != 0:
        sums.append(matrix[n_pairs])
    
    # Flatten the resulting list of sums
    return [num for row in sums for num in row]

def pairwise_sum(matrix):
    summed_3d_np = np.array([sum_np(layer) for layer in matrix]) / 2
    return summed_3d_np

In [ ]:
y_pred_final = pairwise_sum(y_pred)
y_test_final = pairwise_sum(y_test)

In [ ]:
print(y_test_final.shape)
print(y_pred_final.shape)
mse = mean_squared_error(y_test_final, y_pred_final)
rmse = math.sqrt(mse)
mae = mean_absolute_error(y_test_final, y_pred_final)
r2 = r2_score(y_test_final, y_pred_final)
print("-" * 86)
print(f'mse: {mse:.4f}')
print(f'rmse: {rmse:.4f}')
print(f'mae: {mae:.4f}')
print(f'r2: {r2:.4f}')
print("-" * 86)

In [ ]:
TEST_PLOT_DIR = "./test_plots/bi_nn_96steps/"

In [ ]:
if not os.path.exists(TEST_PLOT_DIR):
    os.makedirs(TEST_PLOT_DIR)
if not os.path.exists("./model"):
    os.makedirs("./model")

In [ ]:
pred_data = scaler.inverse_transform(y_pred_final)
actual_data = scaler.inverse_transform(y_test_final)
previous_data = scaler.inverse_transform(X_test_b_flatten)

for i in range(actual_data.shape[0]):
    history = None
    history_timestamp = None
    index = i % 192
    if index < 47:
        history = pred_data[i-index:i+1, 0]
        history_timestamp = y_test_timestamp[i-index:i+1, 0]
    else:
        history = pred_data[i-48:i+1, 0]
        history_timestamp = y_test_timestamp[i-48:i+1, 0]
        
    plt.figure(figsize=(12, 6))
    X1 = np.concatenate((X_test_b_timestamp[i][-48:], y_test_timestamp[i]))
    y1 = np.concatenate((previous_data[i][-48:], actual_data[i]))
    X2 = y_test_timestamp[i]
    y_p = pred_data[i]
    y_a = actual_data[i]
    Xh = np.full(100, X1[len(X1)-96])
    yh = np.arange(0, 100, 1)
    plt.title(f"Time Series {i+1} prediction")
    plt.plot(X1, y1, '.-', label='Actual', color='#6eb5c0')
    plt.plot(X2, y_p, '.-', label='Predict', color='#174d7c')
    plt.plot(history_timestamp, history, '--', label='History', color='#989898')
    plt.plot(Xh, yh, color='#4863a0', alpha=0.5)
    plt.ylim(0, 100)
    plt.xlabel('Time step')
    plt.ylabel('Usage (kWh)')
    plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
    plt.tight_layout()
    plt.savefig(TEST_PLOT_DIR+f"Time_Series_{i+1}.png")
    plt.close()

In [ ]:
model.save("./model/image_inpainting_CNN_LSTM_96steps.keras") 